In [39]:
import json
import numpy as np
import pandas as pd

In [54]:
dbPedia = json.load(open("Data/dbPedia_extracted_demonyms3.json"))
infobox = json.load(open("Data/infobox_extracted_demonyms3.json"))
habitants_com =  json.load(open("Data/habitants_com_extracted_demonyms3.json"))

In [55]:
regions = json.load(open("Data/scrapped/regions_demonyms.json"))

In [56]:
df=pd.read_csv("Data/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
insee_communes, counts = np.unique(communes_, return_counts=True)

In [61]:
len(communes_), len(insee_communes)

(37742, 35084)

In [64]:
nb = 0
nb2 = 0
nb3 = 0
nb4 = 0
for c in insee_communes:
    if c not in habitants_com and (c in infobox or c in dbPedia):
        nb+=1
    if c in habitants_com and (c  not in infobox and c not in dbPedia):
        nb2+=1
    if c not in habitants_com and c  not in infobox and c not in dbPedia:
        nb3+=1
    if c in habitants_com or c  in infobox or c in dbPedia:
        nb4+=1
nb, nb2, nb3, nb4

(3842, 6607, 5572, 29512)

In [65]:
len(insee_communes) - len(habitants_com) - nb

2803

In [62]:
35084 - 29512

5572